https://medicalxpress.com/

In [11]:
# !pip install urllib3==1.25.11

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
selenium 4.10.0 requires urllib3[socks]<3,>=1.26, but you have urllib3 1.25.11 which is incompatible.



  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9


In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}


In [1]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    res = requests.get('https://www.medicinenet.com/', headers=headers_pc)
    # 将cookieJar数据转化为字典，
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

# 1、先获取Topics术语链接列表

In [ ]:
headers_pc, randomproxy, cookie_dict = changeparams()

results = []

topic_url = 'https://medicalxpress.com/'
r = requests.get(topic_url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
r.encoding = 'utf-8'
html = r.text
with open(f'D:/Intern_Project/12_medicalxpress/topics列表页.html', 'w+', encoding='utf-8') as f:
    f.write(html)

selector = Selector(text=html)
termUrl_list = selector.xpath(
    "//ul[@class='header-articles-nav']/li[@class='nav-item']/a")

for termUrl in termUrl_list:
    tmp_url = termUrl.xpath('./@href').extract()[0]
    tmp_term = termUrl.xpath('.//text()').extract()[0]
    # print(term)
    tmp = {
        'term': tmp_term,
        'url': tmp_url
    }
    results.append(tmp)

# 2、获取Conditions术语链接列表

In [ ]:
def get_conditions_list(url, headers_pc, randomproxy, cookie_dict):
    letter_result = []
    try:
        requests.adapters.DEFAULT_RETRIES = 20 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text
#         if os.path.exists(f"D:/Intern_Project/8_RxList/{tag}_{letter}列表页.html"):   
#             return None
#         with open(f'D:/Intern_Project/10_MedicineNet/列表页/{tag}_{letter}列表页.html', 'w+', encoding='utf-8') as f:
#              f.write(html)

        selector = Selector(text=html)
        termUrl_list = selector.xpath(
            "//div[@class='container']/div[@class='text-column-4 mb-5']/p/a")
        #print(len(termUrl_list))
        for termUrl in termUrl_list:
            tmp_url = termUrl.xpath('./@href').extract()[0]
            tmp_term = termUrl.xpath('.//text()').extract()[0]
            # print(term)
            tmp = {
                'term': tmp_term,
                'url': tmp_url
            }
            letter_result.append(tmp)
            # time.sleep(1)
        # pbar.update(1)
    except Exception as e:
        print(e)
        time.sleep(3)
        headers_pc, randomproxy, cookie_dict = changeparams()
        get_conditions_list(url, headers_pc, randomproxy, cookie_dict)
        mislist.append(url)
    return letter_result

In [ ]:
headers_pc, randomproxy, cookie_dict = changeparams()
letters = [chr(i) for i in range(97, 123)] # 生成26个英文字母的列表
mislist = []
for letter in letters:
    conditions_url = f'https://medicalxpress.com/conditions/?fl={letter}'
    letter_result = get_conditions_list(conditions_url, headers_pc, randomproxy, cookie_dict)
    time.sleep(3)
    results.extend(letter_result)

In [9]:
print(len(results))
print(mislist)

627
[]


In [10]:
with open(f'D:/Intern_Project/12_medicalxpress/术语链接列表.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(results, indent=2, ensure_ascii=False))

# 3、开始获取每个术语全部文章的链接

In [2]:
with open(f'D:/Intern_Project/12_medicalxpress/术语链接列表.json', 'r', encoding='utf-8') as f:
    results = json.load(f)

In [3]:
# 拆分一下json文件
results1 = results[:200]
results2 = results[200:400]
results3 = results[400:]

with open(f'D:/Intern_Project/12_medicalxpress/术语链接列表0_199.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(results1, indent=2, ensure_ascii=False))
with open(f'D:/Intern_Project/12_medicalxpress/术语链接列表200_399.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(results2, indent=2, ensure_ascii=False))
with open(f'D:/Intern_Project/12_medicalxpress/术语链接列表400_627.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(results3, indent=2, ensure_ascii=False))

In [5]:
def get_article_url_list(url, headers_pc, randomproxy, cookie_dict, term, i):
    article_result = []
    try:
        requests.adapters.DEFAULT_RETRIES = 20 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text
#         if os.path.exists(f"D:/Intern_Project/8_RxList/{tag}_{letter}列表页.html"):   
#             return None
        with open(f'D:/Intern_Project/12_medicalxpress/列表页/{term}_{i}列表页.html', 'w+', encoding='utf-8') as f:
             f.write(html)
        
        # TODO：加个判断，如果文件大小小于2kb，也把丢失页面记录下来
        size = round(os.path.getsize(f'D:/Intern_Project/12_medicalxpress/列表页/{term}_{i}列表页.html')/float(1024),2)
        if size<2.0:
            # time.sleep(random.randint(3,5))
            headers_pc, randomproxy, cookie_dict = changeparams()
            get_article_url_list(url, headers_pc, randomproxy, cookie_dict)
            mis_article_list.append(url)
        
        
        
        # TODO：为了提高效率，先下载下来，之后再解析？

        selector = Selector(text=html)
        titleUrl_list = selector.xpath(
            "//div[@class='sorted-article-content d-flex flex-column ie-flex-1']/h2/a")
        #print(len(termUrl_list))
        for titleUrl in titleUrl_list:
            tmp_url = titleUrl.xpath('./@href').extract()[0]
            tmp_title = titleUrl.xpath('.//text()').extract()[0]
            # print(term)
            tmp = {
                'title': tmp_title,
                'url': tmp_url
            }
            article_result.append(tmp)
            # time.sleep(1)
        # pbar.update(1)
    except Exception as e:
        print(e)
        # time.sleep(random.randint(3,5))
        headers_pc, randomproxy, cookie_dict = changeparams()
        get_article_url_list(url, headers_pc, randomproxy, cookie_dict)
        mis_article_list.append(url)
    return article_result

In [6]:
def task_monitor(task_list, mt_pool, results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
                results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [7]:
def get_pagenum(url, headers_pc, randomproxy, cookie_dict):
    try:
        requests.adapters.DEFAULT_RETRIES = 20 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        html = r.text
#         if os.path.exists(f"D:/Intern_Project/8_RxList/{tag}_{letter}列表页.html"):   
#             return None
#         with open(f'D:/Intern_Project/10_MedicineNet/列表页/{tag}_{letter}列表页.html', 'w+', encoding='utf-8') as f:
#              f.write(html)

        selector = Selector(text=html)
        pagenum = selector.xpath(
            "//div[@class='pagination-view mr-4']/span/text()").extract()[0]
        pagenum = int(pagenum)
    except Exception as e:
        print(e)
        # time.sleep(random.randint(3,5))
        headers_pc, randomproxy, cookie_dict = changeparams()
        get_pagenum(url, headers_pc, randomproxy, cookie_dict)
    return pagenum

In [ ]:
mis_item_list = []
mis_article_list = []

all_craw_tasks = []
article_results = []

headers_pc, randomproxy, cookie_dict = changeparams()

# pbar = tqdm(total=len(results))
with ThreadPoolExecutor(max_workers=10) as mt_pool:
    for item in results:
        pagenum = 0
        headers_pc, randomproxy, cookie_dict = changeparams()
        start_url = (item['url'] + 'sort/date/all/')
        start_url.replace('https','http')
        term = item['term']
        try:
            pagenum = get_pagenum(start_url, headers_pc, randomproxy, cookie_dict)
        except Exception as e:
            print(e)
            mis_item_list.append(item)
        for i in range(pagenum):
            page_url = item['url'] + 'sort/date/all/' + f'page{i}.html'
            page_url.replace('https','http')
            tmp_task = mt_pool.submit(get_article_url_list, page_url, headers_pc, randomproxy, cookie_dict, term , i)
            all_craw_tasks.append(tmp_task)
            if i % 5 == 0:
                time.sleep(random.randint(3,5))
                headers_pc, randomproxy, cookie_dict = changeparams()
        # pbar.update(1)
    
    task_monitor(all_craw_tasks, mt_pool, article_results)
    
    with open(f'D:/Intern_Project/12_medicalxpress/文章链接列表.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(article_results, indent=2, ensure_ascii=False))

HTTPSConnectionPool(host='medicalxpress.com', port=443): Max retries exceeded with url: /addiction-news/sort/date/all/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
HTTPSConnectionPool(host='medicalxpress.com', port=443): Max retries exceeded with url: /addiction-news/sort/date/all/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
HTTPSConnectionPool(host='medicalxpress.com', port=443): Max retries exceeded with url: /addiction-news/sort/date/all/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))
HTTPSConnectionPool(host='medicalxpress.com', port=443): Max retries exceeded with url: /addiction-news/sort/date/all/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))


# 用来测试哪些页面能简单获取到

In [6]:

headers_pc, randomproxy, cookie_dict = changeparams()
            
requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
s = requests.session()
s.keep_alive = False # 关闭多余连接

# url = f'https://www.medicinenet.com/diseases_and_conditions/alpha_u.htm'
start_url = f'https://medicalxpress.com/alzheimer-dementia-news/' + 'sort/date/all/'
# r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
try:
    pagenum = get_pagenum(start_url, headers_pc, randomproxy, cookie_dict)

except Exception as e:
    print(e)

print(pagenum)

497


In [7]:
with open(f'D:/Intern_Project/12_medicalxpress/术语链接列表.json', 'r', encoding='utf-8') as f:
    results = json.load(f)

In [ ]:
allarticle_result = []
for i in range(pagenum):
    page_url = 'https://medicalxpress.com/alzheimer-dementia-news/' + 'sort/date/all/' + f'page{i}.html'
    term = 'alzheimer-dementia-news'
    allarticle_result.extend(get_article_url_list(page_url, headers_pc, randomproxy, cookie_dict, term , i))
    if i % 5 == 0:
        time.sleep(2)
        headers_pc, randomproxy, cookie_dict = changeparams()

In [ ]:
article_result

In [9]:
r = requests.get('http://www.baidu.com', headers = headers_pc)